In [9]:
import random
import pandas as pd
from glob import glob
import numpy as np  
import os

In [10]:
def train_validate_test_split(img_dir, class_label=0, train_percent=.4, validate_percent=.1, base_train_percent=.4, seed=87264):
    img_list = glob(img_dir + '*.dcm')
    img_list = [os.path.join(s.split('/')[-2], s.split('/')[-1]) for s in img_list] #this is really ugly!
    df = pd.DataFrame({'image':img_list, 'label':[class_label]*len(img_list), 'split':['-']*len(img_list)})
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    base_train_end = int(base_train_percent * m) + train_end
    validate_end = int(validate_percent * m) + base_train_end
    df.loc[perm[:train_end], 'split'] = 'train'
    df.loc[perm[train_end:base_train_end], 'split'] = 'base_train'
    df.loc[perm[base_train_end:validate_end], 'split'] = 'val'
    df.loc[perm[validate_end:], 'split'] = 'test'
    return df

In [11]:
df_hr = train_validate_test_split('/project/catinous/cat_data/hr/', class_label=0)
df_hr_cat = train_validate_test_split('/project/catinous/cat_data/hr_cat/', class_label=1)
df_hr = df_hr.append(df_hr_cat)
df_hr['res'] = 'hr'
df_hr.to_csv('catsinom_hr_dataset.csv')

In [12]:
df_lr = train_validate_test_split('/project/catinous/cat_data/lr/', class_label=0)
df_lr_cat = train_validate_test_split('/project/catinous/cat_data/lr_cat/', class_label=1)
df_lr = df_lr.append(df_lr_cat)
df_lr['res'] = 'lr'
df_lr.to_csv('catsinom_lr_dataset.csv')

In [13]:
df = df_lr.append(df_hr, sort=False)
df.to_csv('catsinom_combined_dataset.csv')

In [14]:
df_hr = train_validate_test_split('/project/catinous/cat_data/hr/', class_label=0)
df_hr_cat_ls = train_validate_test_split('/project/catinous/cat_data/hr_cat_lowshift/', class_label=1)
df_hr = df_hr.append(df_hr_cat_ls)
df_hr['res'] = 'hr'
df = df_lr.append(df_hr, sort=False)
df.to_csv('catsinom_combined_hrlowshift_dataset.csv')

In [29]:
df.sample(100, replace=False).to_csv('test_catsinom_combined_dataset.csv')

In [19]:
df

,image,label,split,res
0,lr/D4EDA423_20040121_dicom_38786.dcm,0,train,lr
1,lr/29522EA0_20101128_dicom_8439.dcm,0,val,lr
2,lr/AE6E3CE1_20150623_dicom_13352.dcm,0,train,lr
3,lr/02ZIRSCG_20140826_dicom_192535.dcm,0,base_train,lr
4,lr/27956E43_20100823_dicom_38760.dcm,0,base_train,lr
...,...,...,...,...
2110,hr_cat_lowshift/1D683786_20141006_dicom_61052.dcm,1,train,hr
2111,hr_cat_lowshift/1JDHQRCA_20141027_dicom_233551...,1,test,hr
2112,hr_cat_lowshift/E227C188_20120615_dicom_86896.dcm,1,train,hr
2113,hr_cat_lowshift/47048ACE_20080408_dicom_62543.dcm,1,train,hr


In [16]:
df.sample(10000, replace=True)

,image,label,split,res
2101,hr/1JDGIABZ_20140819_dicom_202198.dcm,0,train,hr
1319,hr/1GE97R84_20130902_dicom_292609.dcm,0,val,hr
1583,lr/BD6B8048_20070425_dicom_65820.dcm,0,base_train,lr
1971,hr/786B4F46_20130912_dicom_68302.dcm,0,val,hr
1750,hr/1FT2FM85_20140403_dicom_268358.dcm,0,train,hr
...,...,...,...,...
1335,hr_cat_lowshift/6661DB81_20070912_dicom_103059...,1,test,hr
1559,lr/A25ED64F_20131231_dicom_37095.dcm,0,test,lr
413,hr/16001CB5_20080331_dicom_58072.dcm,0,base_train,hr
1790,lr_cat/AC654E04_20110406_dicom_90651.dcm,1,base_train,lr


In [8]:
df.shape

(8033, 3)